<a href="https://colab.research.google.com/github/prbsh9/Suspicious-Activity-Detection-from-videos/blob/master/Big_RNNImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and syncing with drive

In [0]:
from google.colab import drive
drive.mount('/drive')
import os
path = '/drive/My Drive/suspicious_activity/content/frameEasy'
os.chdir(path)
os.getcwd()

In [0]:
!pip install livelossplot

In [0]:

!pip install efficientnet

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop, SGD, Adam
from keras.layers import Dense, Flatten, GlobalMaxPooling2D, Dropout
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback
import efficientnet.keras as efn
import keras
from keras.layers import Activation
from keras.layers import LSTM, CuDNNLSTM
from keras.preprocessing import image
import numpy as np
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint


# RNN part?

###  Importing from CSV

In [0]:
x_train = np.load('x_train.npy') 
y_train = np.load('y_train.npy') 

x_test = np.load('x_valid.npy')
y_test = np.load('y_valid.npy')

In [0]:
# x_train = x_train/255.0
# x_test = x_test/255.0

In [0]:
train_data = x_train
validation_data = x_test

In [0]:
train_data = train_data.reshape(train_data.shape[0],
                     train_data.shape[1] * train_data.shape[2],
                     train_data.shape[3])
validation_data = validation_data.reshape(validation_data.shape[0],
                     validation_data.shape[1] * validation_data.shape[2],
                     validation_data.shape[3])

In [0]:
# LSTM
model = Sequential()
model.add(CuDNNLSTM(2048, input_shape=(train_data.shape[1:]), return_sequences=True ))

model.add(CuDNNLSTM(1024))

model.add(Dense(2024, activation='relu'))
model.add(Dropout(0.1))


model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# sgd = SGD(lr=0.00005, decay = 1e-6, momentum=0.9)
sgd = Adam(lr=0.0005, decay = 1e-5)

model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), ModelCheckpoint('video_1_LSTM_1_1024.h5', monitor='val_loss', save_best_only=True, verbose=0) ]
nb_epoch = 500
batch_size = 64

model.fit(train_data,y_train,validation_data=(validation_data,y_test),batch_size=batch_size,nb_epoch=nb_epoch,callbacks=callbacks,shuffle=True,verbose=1)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':





Train on 6489 samples, validate on 834 samples
Epoch 1/500





  64/6489 [..............................] - ETA: 25:52 - loss: 0.6918 - acc: 0.5625

In [0]:
data =[]
target=[]
epoch = 100
batchS = 100
for i in joint_transfer:
    data.append(i)
    target.append(np.array(i[1]))
model.fit(data, target, epochs=epoch, batch_size=batchS, verbose=1)

NameError: ignored

In [0]:

  
# c.shape[0]

In [0]:
for a in os.listdir('data/valid/Abnorm'):
  npy = np.load(a)
  

  

In [0]:
def train(data_type='features', seq_length=40, model='lstm', saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
  

  # Get the data and process it.
      data = DataSet(seq_length=seq_length, class_limit=class_limit)


  # Get samples per epoch.
  # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
  steps_per_epoch = (len(data.data) * 0.7) // batch_size

  if load_to_memory:
      # Get data.
      X, y = data.get_all_sequences_in_memory('train', data_type)
      X_test, y_test = data.get_all_sequences_in_memory('test', data_type)
  else:
      # Get generators.
      generator = data.frame_generator(batch_size, 'train', data_type)
      val_generator = data.frame_generator(batch_size, 'test', data_type)

  # Get the model.
  rm = LSTM()

      # Use fit generator.
  rm.model.fit_generator(
      generator=generator,
      steps_per_epoch=steps_per_epoch,
      epochs=nb_epoch,
      verbose=1,
      callbacks=[tb, early_stopper, csv_logger, checkpointer],
      validation_data=val_generator,
      validation_steps=40,
      workers=4)


In [0]:
# Testing
probabilities = model.predict_generator(test_generator, TEST_SIZE)
for index, probability in enumerate(probabilities):
    image_path = test_data_dir + "/" +test_generator.filenames[index]
    img = mpimg.imread(image_path)
    plt.imshow(img)
    if probability > 0.5:
        plt.title("%.2f" % (probability[0]*100) + "% dog")
    else:
        plt.title("%.2f" % ((1-probability[0])*100) + "% cat")
    plt.show()